<a href="https://colab.research.google.com/github/Aggregate-Intellect/xir/blob/main/wikiQA_fineturning_f1_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beir
!pip install tensorflow-text
!pip install farm-haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install tensorflow

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Load wikiQA dataset

In [ ]:
from datasets import load_dataset

wikiqa_data = load_dataset(
    'wiki_qa',
    split='train',
)
next(iter(wikiqa_data))

Using custom data configuration default


Generating test split:   0%|          | 0/6165 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2733 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20360 [00:00<?, ? examples/s]

Dataset wiki_qa downloaded and prepared to /root/.cache/huggingface/datasets/wiki_qa/default/0.1.0/d2d236b5cbdc6fbdab45d168b4d678a002e06ddea3525733a24558150585951c. Subsequent calls will reuse this data.


{'answer': 'A partly submerged glacier cave on Perito Moreno Glacier .',
 'document_title': 'Glacier cave',
 'label': 0,
 'question': 'how are glacier caves formed?',
 'question_id': 'Q1'}

In [ ]:
import json
import pandas as pd

pd_wikiqa_data = pd.DataFrame(wikiqa_data)
pd_wikiqa_data

,question_id,question,document_title,answer,label
0,Q1,how are glacier caves formed?,Glacier cave,A partly submerged glacier cave on Perito More...,0
1,Q1,how are glacier caves formed?,Glacier cave,The ice facade is approximately 60 m high,0
2,Q1,how are glacier caves formed?,Glacier cave,Ice formations in the Titlis glacier cave,0
3,Q1,how are glacier caves formed?,Glacier cave,A glacier cave is a cave formed within the ice...,1
4,Q1,how are glacier caves formed?,Glacier cave,"Glacier caves are often called ice caves , but...",0
...,...,...,...,...,...
20355,Q3043,what is section eight housing,Section 8 (housing),A tenant who leaves a subsidized project will ...,0
20356,Q3043,what is section eight housing,Section 8 (housing),The United States Department of Housing and Ur...,0
20357,Q3044,what is the main type of restaurant,Category:Types of restaurants,Restaurants categorized by type and informatio...,0
20358,Q3046,what is us dollar worth based on,History of the United States dollar,U.S. Federal Reserve notes in the mid-1990s,0


# convert to squad-like format

In [ ]:
data = []
for question_id in pd_wikiqa_data['question_id'].unique().tolist():
        pd_question = pd_wikiqa_data.loc[pd_wikiqa_data['question_id'] == question_id]   
        context = ' '.join(pd_question["answer"].tolist())
        pd_answer = pd_question[pd_question['label'] == 1]
        if not pd_answer.empty:
            answer = pd_answer['answer'].tolist()[0]
            para = {'context': context, 'qas': [{'question': pd_question["question"].iloc[0], 'answers': []}]}
            data.append({'title': pd_question['document_title'].iloc[0],'paragraphs': [para]})
            qa = para['qas'][0]  
            qa['id'] = question_id
            index = context.index(answer)
            qa['answers'].append({'text': answer, 'answer_start': index})
            qa['is_impossible'] = False

In [ ]:
wiki_as_squad = {'data': data, 'version': '2.0'}
with open("converted_wikiqa.json", 'w', encoding='utf-8') as outfile:
  json.dump(wiki_as_squad, outfile, indent=2, sort_keys=True, ensure_ascii=False)

# Fine turning on reader's model

In [ ]:
from haystack.nodes import FARMReader
reader = FARMReader(model_name_or_path="distilbert-base-uncased-distilled-squad", use_gpu=True)
data_dir = "/content"
reader.train(data_dir=data_dir, train_filename="converted_wikiqa.json", use_gpu=True, n_epochs=1, save_dir="my_model")

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find distilbert-base-uncased-distilled-squad locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/451 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/253M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded distilbert-base-uncased-distilled-squad


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0  
INFO - haystack.modeling.infer -  /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \ 
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: /content/converted_wikiqa.json 
INFO - haystack.modeling.data_handler.data_silo -  Multiprocessing disabled, using a single wo

# Preprocessing dataset and prepare document store

In [ ]:
from typing import List
import requests
import pandas as pd
from haystack import Document
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import RAGenerator, DensePassageRetriever
from haystack.utils import fetch_archive_from_http, print_answers, print_documents

from haystack.document_stores import FAISSDocumentStore
from haystack import Document

from haystack.modeling.evaluation.squad import compute_f1 as calculate_f1_str
from haystack.modeling.evaluation.squad import compute_exact as calculate_em_str

pd_data = pd.DataFrame(wikiqa_data)

# make sure all sentences ending with .
# if last character is not . ? !, add . at the end
for index in range(0,pd_data.shape[0]):
  last_char = pd_data['answer'][index][-1]
  if not (last_char =='.' or last_char =='!' or last_char =='?'):
    pd_data['answer'][index] = pd_data['answer'][index] + '.'

pd_data.to_csv("wikiQA_dataset.csv", index=False)

processed_wikiqa = pd.DataFrame(columns = ["question_id", "question", "document_title", "document_content", "answer"])
processed_all_wikiqa = pd.DataFrame(columns = ["question_id", "document_title", "document_content"])

for question in pd_data['question_id'].unique().tolist():
    pd_question = pd_data.loc[pd_data['question_id'] == question]   
    content = ' '.join(pd_question["answer"].tolist())
    pd_answer = pd_question[pd_question['label'] == 1]
    processed_all_wikiqa = processed_all_wikiqa.append({"question_id": question,
                                                        "document_title": pd_question["document_title"].iloc[0],
                                                        "document_content" : content}, ignore_index=True)
    if not pd_answer.empty:
        answer = ' '.join(pd_answer['answer'].tolist())
        new_row = {"question_id": pd_question["question_id"].iloc[0], 
                  "question" : pd_question["question"].iloc[0],
                  "document_title" : pd_question["document_title"].iloc[0],
                  "document_content" : content,
                  "answer": answer}
        processed_wikiqa = processed_wikiqa.append(new_row, ignore_index=True)
processed_wikiqa.to_csv("processed_wikiqa.csv", index=False)

docs = []
for index, row in processed_all_wikiqa.iterrows():
    # create haystack document object with text content and doc metadata
    doc = Document(
        content = row['document_content'],
        meta = {
            "document_title": row['document_title']
        }
    )
    docs.append(doc)

document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)

# Delete existing documents in documents store
document_store.delete_documents()

document_store.write_documents(docs)
document_store.get_document_count()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Writing Documents:   0%|          | 0/2118 [00:00<?, ?it/s]

1994

# retriever and update embeddings

In [ ]:
from haystack.retriever.dense import EmbeddingRetriever

retriever = EmbeddingRetriever(
   document_store=document_store,
   embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
   model_format="sentence_transformers"
)

document_store.update_embeddings(retriever, batch_size=128)

# performance of original reader model

In [ ]:
from haystack.nodes import FARMReader, TransformersReader
reader = FARMReader(context_window_size = 130,model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

from haystack.pipelines import ExtractiveQAPipeline
from haystack.pipelines import DocumentSearchPipeline

IR_pipeline = DocumentSearchPipeline(retriever = retriever)
reader_pipe = ExtractiveQAPipeline(reader, retriever)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0  
INFO - haystack.modeling.infer -  /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \ 


In [ ]:
processed_wikiqa

In [ ]:
number_documents = 873
f1_total = 0

for index, row in processed_wikiqa.iterrows():
    query = row['question']
    result = reader_pipe.run(
             query=query, 
             params={"Retriever": {"top_k": 1}}
             )

    prediction = result['answers'][0].context
    true_answer = processed_wikiqa['answer'][index]

    f1 = calculate_f1_str(true_answer, prediction)
    f1_total += f1 

    if index == number_documents - 1:
        break

In [ ]:
average_f1 = f1_total / number_documents
print("f1: ", average_f1)

f1:  0.5449653068595183


# performance of fine-turning reader model

In [ ]:
from haystack.nodes import FARMReader, TransformersReader
new_reader = FARMReader(model_name_or_path="my_model")


from haystack.pipelines import ExtractiveQAPipeline
from haystack.pipelines import DocumentSearchPipeline

IR_pipeline = DocumentSearchPipeline(retriever = retriever)
new_reader_pipe = ExtractiveQAPipeline(new_reader, retriever)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at my_model
INFO - haystack.modeling.model.language_model -  Loaded my_model
INFO - haystack.modeling.model.adaptive_model -  Found files for loading 1 prediction heads
WARNING - haystack.modeling.model.prediction_head -  Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": true, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}
INFO - haystack.modeling.model.prediction_head -  Loading prediction head from my_model/prediction_head_0.bin
INFO - hay

In [ ]:
number_documents = 873
f1_total = 0

for index, row in processed_wikiqa.iterrows():
    query = row['question']
    result = new_reader_pipe.run(
             query=query, 
             params={"Retriever": {"top_k": 1}}
             )

    prediction = result['answers'][0].context
    true_answer = processed_wikiqa['answer'][index]

    f1 = calculate_f1_str(true_answer, prediction)
    f1_total += f1 

    if index == number_documents - 1:
        break

In [ ]:
average_f1 = f1_total / number_documents
print("f1: ", average_f1)

f1:  0.678656752255021
